# Neural Networks
Necessary imports and plot functions used over the entire python notebook:

In [1]:
import numpy as np
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns

# To plot figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
IMAGES_PATH = "./images"

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## Data Retrieval

In [2]:
from six.moves import urllib
from sklearn.datasets import fetch_mldata
try:
    mnist = fetch_mldata('MNIST original')
except urllib.error.HTTPError as ex:
    print("Could not download MNIST data from mldata.org, trying alternative...")

    # Alternative method to load MNIST, if mldata.org is down
    from scipy.io import loadmat
    mnist_alternative_url = "https://github.com/amplab/datascience-sp14/raw/master/lab7/mldata/mnist-original.mat"
    mnist_path = "./mnist-original.mat"
    response = urllib.request.urlopen(mnist_alternative_url)
    with open(mnist_path, "wb") as f:
        content = response.read()
        f.write(content)
    mnist_raw = loadmat(mnist_path)
    mnist = {
        "data": mnist_raw["data"].T,
        "target": mnist_raw["label"][0],
        "COL_NAMES": ["label", "data"],
        "DESCR": "mldata.org dataset: mnist-original",
    }

In [3]:
X, y = mnist["data"]/255.0, mnist["target"]
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

## Neural Network
We have the following architecture for the neural network.
1. CNN layer: 20 nodes, kernel size=5
2. Max pooling layer: kernel size=2, stride=2
3. CNN layer: 50 nodes, kernel size=5
4. Max Pooling layer: kernel size=2, stride=2
5. Fully Connected Layer: 500 nodes
6. ReLU (Rectified Linear Unit)
7. Output Layer: 10 nodes

In [6]:
from tensorflow import keras
# from keras.models import Sequential
# from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
# from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

img_rows, img_cols = 28, 28

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

print ('X_train:', X_train.shape)
print ('X_test:', X_test.shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

ModuleNotFoundError: No module named 'tensorflow'

In [6]:
model = Sequential()
model.add(Conv2D(20, kernel_size=5, input_shape=input_shape))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Conv2D(50, kernel_size=5))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

print(model.summary())

X_train: (60000, 28, 28, 1)
X_test: (10000, 28, 28, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 24, 24, 20)        520       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 20)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 50)          25050     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 4, 50)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 500)               400500    
_________________________________________________________________
dense_4 (Dense)      

In [5]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 18s - loss: 0.2421 - acc: 0.9291 - val_loss: 0.0851 - val_acc: 0.9720
Epoch 2/12
60000/60000 [==============================] - 18s - loss: 0.0659 - acc: 0.9799 - val_loss: 0.0495 - val_acc: 0.9841
Epoch 3/12
60000/60000 [==============================] - 18s - loss: 0.0442 - acc: 0.9860 - val_loss: 0.0400 - val_acc: 0.9874
Epoch 4/12
60000/60000 [==============================] - 18s - loss: 0.0329 - acc: 0.9901 - val_loss: 0.0354 - val_acc: 0.9891
Epoch 5/12
60000/60000 [==============================] - 18s - loss: 0.0255 - acc: 0.9925 - val_loss: 0.0348 - val_acc: 0.9886
Epoch 6/12
60000/60000 [==============================] - 18s - loss: 0.0202 - acc: 0.9939 - val_loss: 0.0318 - val_acc: 0.9888
Epoch 7/12
60000/60000 [==============================] - 18s - loss: 0.0156 - acc: 0.9954 - val_loss: 0.0356 - val_acc: 0.9892
Epoch 8/12
60000/60000 [==============================

## Other network Architectures: increasing no. of nodes

In [6]:
model = Sequential()
model.add(Conv2D(30, kernel_size=5, input_shape=input_shape))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Conv2D(50, kernel_size=5))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 71s - loss: 0.2319 - acc: 0.9315 - val_loss: 0.0813 - val_acc: 0.9738
Epoch 2/12
60000/60000 [==============================] - 74s - loss: 0.0635 - acc: 0.9809 - val_loss: 0.0524 - val_acc: 0.9839
Epoch 3/12
60000/60000 [==============================] - 73s - loss: 0.0421 - acc: 0.9873 - val_loss: 0.0370 - val_acc: 0.9883
Epoch 4/12
60000/60000 [==============================] - 74s - loss: 0.0313 - acc: 0.9905 - val_loss: 0.0379 - val_acc: 0.9874
Epoch 5/12
60000/60000 [==============================] - 73s - loss: 0.0233 - acc: 0.9930 - val_loss: 0.0357 - val_acc: 0.9879
Epoch 6/12
60000/60000 [==============================] - 72s - loss: 0.0186 - acc: 0.9941 - val_loss: 0.0326 - val_acc: 0.9896
Epoch 7/12
60000/60000 [==============================] - 72s - loss: 0.0140 - acc: 0.9957 - val_loss: 0.0341 - val_acc: 0.9889
Epoch 8/12
60000/60000 [==============================

In [7]:
model = Sequential()
model.add(Conv2D(30, kernel_size=5, input_shape=input_shape))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Conv2D(60, kernel_size=5))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Flatten())
model.add(Dense(600, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 80s - loss: 0.2204 - acc: 0.9342 - val_loss: 0.0748 - val_acc: 0.9775
Epoch 2/12
60000/60000 [==============================] - 80s - loss: 0.0598 - acc: 0.9822 - val_loss: 0.0415 - val_acc: 0.9867
Epoch 3/12
60000/60000 [==============================] - 80s - loss: 0.0402 - acc: 0.9872 - val_loss: 0.0366 - val_acc: 0.9889
Epoch 4/12
60000/60000 [==============================] - 79s - loss: 0.0285 - acc: 0.9914 - val_loss: 0.0297 - val_acc: 0.9903
Epoch 5/12
60000/60000 [==============================] - 80s - loss: 0.0215 - acc: 0.9931 - val_loss: 0.0343 - val_acc: 0.9894
Epoch 6/12
60000/60000 [==============================] - 80s - loss: 0.0168 - acc: 0.9953 - val_loss: 0.0336 - val_acc: 0.9896
Epoch 7/12
60000/60000 [==============================] - 79s - loss: 0.0124 - acc: 0.9964 - val_loss: 0.0311 - val_acc: 0.9907
Epoch 8/12
60000/60000 [==============================

## Other network architectures: Adding layers

In [12]:
model = Sequential()
model.add(Conv2D(20, kernel_size=5, input_shape=input_shape))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Conv2D(30, kernel_size=5))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Conv2D(50, kernel_size=2))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 52s - loss: 0.4392 - acc: 0.8633 - val_loss: 0.1615 - val_acc: 0.9494
Epoch 2/12
60000/60000 [==============================] - 50s - loss: 0.1425 - acc: 0.9568 - val_loss: 0.1170 - val_acc: 0.9620
Epoch 3/12
60000/60000 [==============================] - 52s - loss: 0.1096 - acc: 0.9663 - val_loss: 0.0994 - val_acc: 0.9681
Epoch 4/12
60000/60000 [==============================] - 51s - loss: 0.0902 - acc: 0.9721 - val_loss: 0.0849 - val_acc: 0.9714
Epoch 5/12
60000/60000 [==============================] - 53s - loss: 0.0767 - acc: 0.9769 - val_loss: 0.0779 - val_acc: 0.9743
Epoch 6/12
60000/60000 [==============================] - 53s - loss: 0.0667 - acc: 0.9799 - val_loss: 0.0708 - val_acc: 0.9773
Epoch 7/12
60000/60000 [==============================] - 52s - loss: 0.0587 - acc: 0.9821 - val_loss: 0.0706 - val_acc: 0.9779
Epoch 8/12
60000/60000 [==============================

## Other network architectures: Activation functions

In [13]:
model = Sequential()
# model.add(Conv2D(20, kernel_size=5, input_shape=input_shape, activation='relu'))
# model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Conv2D(30, kernel_size=5, activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Conv2D(50, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 57s - loss: 0.5144 - acc: 0.8337 - val_loss: 0.1556 - val_acc: 0.9527
Epoch 2/12
60000/60000 [==============================] - 56s - loss: 0.1328 - acc: 0.9594 - val_loss: 0.1075 - val_acc: 0.9663
Epoch 3/12
60000/60000 [==============================] - 57s - loss: 0.1007 - acc: 0.9686 - val_loss: 0.1020 - val_acc: 0.9677
Epoch 4/12
60000/60000 [==============================] - 57s - loss: 0.0829 - acc: 0.9745 - val_loss: 0.0844 - val_acc: 0.9738
Epoch 5/12
60000/60000 [==============================] - 57s - loss: 0.0705 - acc: 0.9780 - val_loss: 0.0765 - val_acc: 0.9756
Epoch 6/12
60000/60000 [==============================] - 55s - loss: 0.0620 - acc: 0.9809 - val_loss: 0.0688 - val_acc: 0.9782
Epoch 7/12
60000/60000 [==============================] - 56s - loss: 0.0549 - acc: 0.9831 - val_loss: 0.0640 - val_acc: 0.9803
Epoch 8/12
60000/60000 [==============================